contract store_value(value: int){

    let storage: int = 0;

    entry replace(new_value: int){
       
        storage = new_value;
    } 
}

In [1]:
from lexer.tzscript_lexer import TzScriptLexer
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry
from parser.slr_parser import SLR1Parser, build_slr_ast
words_separated_by_spaces = ['contract', 'store_value','(','value',':', 'int',')','{','let', 'storage',':', 'int','=', '0',';', 'entry', 'replace','(','new_value',':', 'int',')','{','storage', '=', 'new_value',';','}','}']
table = [(TZSCRIPT_GRAMMAR[tok], tok) for tok in words_separated_by_spaces]

expected_tokens = [ Token(x[1], x[0]) for x in table]

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)
tokens = [Token('contract',contract),Token('store_value',idx),Token('(',opar),Token('value',idx),Token(':',colon),Token('int',typex),Token(')',cpar),Token('{',ocur),Token('let',let),Token('storage',idx),Token(':',colon),Token('int',typex),Token('=',equal),Token('0',num),Token(';',semi),Token('entry',entry),Token('replace',idx),Token('(',opar),Token('new_value',idx),Token(':',colon),Token('int',typex),Token(')',cpar),Token('{',ocur),Token('storage',idx),Token('=',equal),Token('new_value',idx),Token(';',semi),Token('}',ccur),Token('}',ccur),Token('EOF',TZSCRIPT_GRAMMAR.EOF)]

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)


0 	 S' -> .<program>, 
	 <program> -> .contractid(<param-list>){<stat_list>},  

1 	 S' -> <program>.,  

2 	 <program> -> contract.id(<param-list>){<stat_list>},  

3 	 <program> -> contractid.(<param-list>){<stat_list>},  

4 	 <param-list> -> .<param>,<param-list>, 
	 <program> -> contractid(.<param-list>){<stat_list>}, 
	 <param> -> .id:type, 
	 <param-list> -> .<param>,  

5 	 <param> -> id.:type,  

6 	 <param> -> id:.type,  

7 	 <param> -> id:type.,  

8 	 <param-list> -> <param>., 
	 <param-list> -> <param>.,<param-list>,  

9 	 <param-list> -> .<param>,<param-list>, 
	 <param> -> .id:type, 
	 <param-list> -> <param>,.<param-list>, 
	 <param-list> -> .<param>,  

10 	 <param-list> -> <param>,<param-list>.,  

11 	 <program> -> contractid(<param-list>.){<stat_list>},  

12 	 <program> -> contractid(<param-list>).{<stat_list>},  

13 	 <def-func> -> .funcid(<param-list>):type{<stat_list>}, 
	 <return-stat> -> .return<expr>;, 
	 <stat> -> .<def-func>, 
	 <stat> -> .<if-stat>, 
	 

In [2]:
productions, operations = derivation

operations


['SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE',
 'REDUCE',
 'REDUCE',
 'SHIFT',
 'REDUCE']

In [3]:
ast = build_slr_ast(productions, operations, tokens)

In [4]:

from visitors.type_check_visitor import TypeCheckVisitor
type_visitor = TypeCheckVisitor()

In [5]:
type_visitor.visit_program(ast)

True

In [6]:
from visitors.scope_check_visitor import ScopeCheckVisitor
scope_visitor = ScopeCheckVisitor()

In [7]:
scope_visitor.visit_program(ast)

True

In [8]:
from visitors.semantic_check_visitor import SemanticCheckVisitor
semantic_visitor = SemanticCheckVisitor()

In [9]:
semantic_visitor.visit_program(ast)

True

In [10]:
# from visitors.string_rep_visitor import StringReprVisitor
# string_visitor = StringReprVisitor(0)

In [11]:
# string_visitor.visit_program(ast)

In [12]:
# print(string_visitor.result)

In [13]:
from visitors.string_rep_visitor import FormatVisitor
format = FormatVisitor()
print(format.visit(ast))

<class 'parser.tzscript_ast.ProgramNode'> tipo
\__ProgramNode: contract node.idx(value : int) [<stat>; ... <stat>;]
	\__AttrDeclarationNode: value : int
		\__VarDeclarationNode: let storage = <expr> : int
			\__ ConstantNumNode: 0
		\__EntryDeclarationNode: Entry replace(new_value : int)
			\__AttrDeclarationNode: new_value : int
				\__VarCallNode: storage = <expr>
					\__ VariableNode: new_value
